In [52]:
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression

In [96]:
saved_df = pd.read_csv("fashion-clip.csv")

In [97]:
len(saved_df)

3705

In [94]:
# delete images with same embeddings

subset = saved_df.drop_duplicates("embeddings").copy()

In [ ]:
len(subset)


# TODO
# Sweatshirt should have mid knit material

3705

In [77]:
count = subset["category"].value_counts()
count

category
Casual T-shirts          1208
Shirts & Blouses         1198
Blazers and Tailoring     625
Knits & Hoodies           435
Tops                      239
Name: count, dtype: int64

In [95]:
# classes = subset["subcategory"].unique()
# classes
count = subset["subcategory"].value_counts()
count
subset.groupby(["category", "subcategory"]).size().reset_index(name="count")


,category,subcategory,count
0,Blazers and Tailoring,Cropped Blazer,101
1,Blazers and Tailoring,Double-Breasted Blazer,48
2,Blazers and Tailoring,Single-Breasted Blazer,469
3,Blazers and Tailoring,Vest,7
4,Casual T-shirts,Henley Shirt,9
5,Casual T-shirts,Oversized T-Shirt,47
6,Casual T-shirts,Polo Shirt,107
7,Casual T-shirts,Regular T-Shirt,879
8,Casual T-shirts,Sleeveless T-Shirt,16
9,Casual T-shirts,Slim-Fit T-Shirt,111


In [79]:

import ast

In [80]:
# image_embeddings = subset["embeddings"].tolist()
image_embeddings = subset["embeddings"].apply(lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else x).tolist()

In [81]:
# fit classificator
classes = subset["fit"].tolist()
fit_X_train, fit_X_test, fit_y_train, fit_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
fit_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(fit_X_train, fit_y_train)
print(classification_report(fit_y_test, fit_clf.predict(fit_X_test)))

               precision    recall  f1-score   support

    Loose Fit       0.23      0.73      0.35        11
Oversized Fit       0.41      0.75      0.53        44
  Regular Fit       0.94      0.66      0.77       387
  Relaxed Fit       0.79      0.94      0.86        63
     Slim Fit       0.43      0.67      0.52        49
 Tailored Fit       0.82      0.90      0.86       187

     accuracy                           0.75       741
    macro avg       0.60      0.77      0.65       741
 weighted avg       0.82      0.75      0.77       741



In [82]:
# length classificator
classes = subset["length"].tolist()
length_X_train, length_X_test, length_y_train, length_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
length_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(length_X_train, length_y_train)
print(classification_report(length_y_test, length_clf.predict(length_X_test)))

              precision    recall  f1-score   support

     Cropped       0.28      0.79      0.42        24
  Hip Length       1.00      0.87      0.93       707
Tunic Length       0.10      0.67      0.17         3
Waist Length       0.10      0.57      0.17         7

    accuracy                           0.86       741
   macro avg       0.37      0.72      0.42       741
weighted avg       0.96      0.86      0.90       741



In [83]:
# front_opening classificator
classes = subset["front_opening"].tolist()
front_opening_X_train, front_opening_X_test, front_opening_y_train, front_opening_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
front_opening_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(front_opening_X_train, front_opening_y_train)
print(classification_report(front_opening_y_test, front_opening_clf.predict(front_opening_X_test)))

              precision    recall  f1-score   support

       False       0.99      0.98      0.98       360
        True       0.98      0.99      0.99       381

    accuracy                           0.99       741
   macro avg       0.99      0.99      0.99       741
weighted avg       0.99      0.99      0.99       741



In [84]:
# neck classificator
class_counts = subset["neck"].value_counts()
neck_subset = subset[subset["neck"].isin(class_counts[class_counts >= 4].index)]
classes = neck_subset["neck"].tolist()
neck_image_embeddings = neck_subset["embeddings"].apply(lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else x).tolist()

neck_X_train, neck_X_test, neck_y_train, neck_y_test = train_test_split(neck_image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
neck_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(neck_X_train, neck_y_train)
print(classification_report(neck_y_test, neck_clf.predict(neck_X_test)))

              precision    recall  f1-score   support

    Collared       0.99      0.98      0.99       260
   Cowl Neck       0.20      1.00      0.33         1
   Crew Neck       0.96      0.83      0.89       297
   Mock Neck       0.27      0.88      0.41         8
     No neck       0.87      0.93      0.90        14
  Scoop Neck       0.76      0.93      0.83        27
  Turtleneck       0.23      0.75      0.35         4
      V-Neck       0.89      0.92      0.90       129

    accuracy                           0.91       740
   macro avg       0.65      0.90      0.70       740
weighted avg       0.94      0.91      0.92       740



In [85]:
# sleeves classificator
classes = subset["sleeves"].tolist()
sleeves_X_train, sleeves_X_test, sleeves_y_train, sleeves_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
sleeves_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(sleeves_X_train, sleeves_y_train)
print(classification_report(sleeves_y_test, sleeves_clf.predict(sleeves_X_test)))

              precision    recall  f1-score   support

 Long Sleeve       0.98      0.98      0.98       396
Short Sleeve       0.98      0.98      0.98       293
  Sleeveless       0.89      0.89      0.89        38
     Strappy       0.79      0.79      0.79        14

    accuracy                           0.97       741
   macro avg       0.91      0.91      0.91       741
weighted avg       0.97      0.97      0.97       741



In [86]:
# fastener classificator
classes = subset["fastener"].tolist()
fastener_X_train, fastener_X_test, fastener_y_train, fastener_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
fastener_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(fastener_X_train, fastener_y_train)
print(classification_report(fastener_y_test, fastener_clf.predict(fastener_X_test)))

                   precision    recall  f1-score   support

             Belt       0.00      0.00      0.00         0
          Buttons       0.97      0.83      0.90       369
Double-Button Row       0.15      0.78      0.25         9
       No Buttons       0.98      0.94      0.96       342
       Tie / Wrap       0.00      0.00      0.00         1
           Zipper       0.36      0.90      0.51        20

         accuracy                           0.88       741
        macro avg       0.41      0.57      0.44       741
     weighted avg       0.95      0.88      0.90       741



/home/cat/.env/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cat/.env/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cat/.env/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cat/.e

In [87]:
# material classificator
classes = subset["material"].tolist()
material_X_train, material_X_test, material_y_train, material_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
material_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(material_X_train, material_y_train)
print(classification_report(material_y_test, material_clf.predict(material_X_test)))

              precision    recall  f1-score   support

  heavy knit       0.72      0.90      0.80        20
 heavy woven       0.98      0.98      0.98       125
  light knit       0.96      0.99      0.97       276
 light woven       0.98      0.95      0.97       253
    mid knit       0.92      0.88      0.90        67

    accuracy                           0.96       741
   macro avg       0.91      0.94      0.93       741
weighted avg       0.96      0.96      0.96       741



In [88]:
# design classificator
classes = subset["design"].tolist()
design_X_train, design_X_test, design_y_train, design_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
design_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(design_X_train, design_y_train)
print(classification_report(design_y_test, design_clf.predict(design_X_test)))

              precision    recall  f1-score   support

       Laces       0.27      1.00      0.43         3
      Peplum       0.38      1.00      0.55         3
         nan       1.00      0.98      0.99       735

    accuracy                           0.98       741
   macro avg       0.55      0.99      0.66       741
weighted avg       0.99      0.98      0.99       741



In [89]:
# hood classificator
classes = subset["hood"].tolist()
hood_X_train, hood_X_test, hood_y_train, hood_y_test = train_test_split(image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
hood_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(hood_X_train, hood_y_train)
print(classification_report(hood_y_test, hood_clf.predict(hood_X_test)))

              precision    recall  f1-score   support

       False       1.00      0.99      0.99       699
        True       0.84      1.00      0.91        42

    accuracy                           0.99       741
   macro avg       0.92      0.99      0.95       741
weighted avg       0.99      0.99      0.99       741



In [90]:
# fit classificator
class_counts = subset["subcategory"].value_counts()
subcategory_subset = subset[subset["subcategory"].isin(class_counts[class_counts >= 4].index)]
classes = subcategory_subset["subcategory"].tolist()
subcategory_image_embeddings = subcategory_subset["embeddings"].apply(lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else x).tolist()

subcategory_X_train, subcategory_X_test, subcategory_y_train, subcategory_y_test = train_test_split(subcategory_image_embeddings, classes, test_size=0.20, random_state=32, stratify=classes)
subcategory_clf = LogisticRegression(random_state=0, class_weight="balanced").fit(subcategory_X_train, subcategory_y_train)
print(classification_report(subcategory_y_test, subcategory_clf.predict(subcategory_X_test)))

                        precision    recall  f1-score   support

       Button-Up Shirt       0.99      0.75      0.85       129
              Camisole       0.79      1.00      0.88        11
              Cardigan       0.33      0.33      0.33         3
        Cropped Blazer       0.46      0.85      0.60        20
Double-Breasted Blazer       0.47      0.70      0.56        10
          Formal Shirt       0.93      0.91      0.92        96
          Henley Shirt       0.50      1.00      0.67         2
Open Cowl Neck Sweater       1.00      1.00      1.00         1
       Oversized Shirt       0.26      1.00      0.41        12
     Oversized T-Shirt       0.29      0.56      0.38         9
            Peplum Top       0.00      0.00      0.00         1
            Polo Shirt       0.77      0.91      0.83        22
       Pullover Hoodie       1.00      0.97      0.99        35
       Regular T-Shirt       0.96      0.81      0.88       176
             Shell Top       0.67      

/home/cat/.env/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cat/.env/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cat/.env/tf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [91]:
import joblib
joblib.dump(design_clf, 'model_design_clf.joblib')
joblib.dump(material_clf, 'model_material_clf.joblib')
joblib.dump(fastener_clf, 'model_fastener_clf.joblib')
joblib.dump(sleeves_clf, 'model_sleeves_clf.joblib')
joblib.dump(neck_clf, 'model_neck_clf.joblib')
joblib.dump(front_opening_clf, 'model_front_opening_clf.joblib')
joblib.dump(length_clf, 'model_length_clf.joblib')
joblib.dump(fit_clf, 'model_fit_clf.joblib')
joblib.dump(hood_clf, 'model_hood_clf.joblib')
joblib.dump(subcategory_clf, 'model_subcategory_clf.joblib')

['model_subcategory_clf.joblib']